# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [139]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np 
from nltk import word_tokenize
from nltk import sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import pickle 
import re 
import nltk
from sklearn.pipeline import Pipeline 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV 

In [140]:

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
#print(stopwords.words('english'))
 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Milo_Sand\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Milo_Sand\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Milo_Sand\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [141]:
# load data from database
engine = create_engine('sqlite:///disaster_project.db')

sql = 'select * from cleaned_messages'
#table is cleaned_messages
df = pd.read_sql_query(sql,engine)
y = df.drop('message',axis=1) 
y = y.drop('id',axis=1) 
y = y.drop('original',axis=1) 
y = y.drop('genre',axis=1)
y = y.astype('int')
#replacing 2s with 0s for related field. 
y['related']=y['related'].replace(2,0)
X = df['message'] 

df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [142]:
#add to X, make dummies. could this improve the model? 
df['genre'].unique()

array(['direct', 'social', 'news'], dtype=object)

### 2. Write a tokenization function to process your text data

In [143]:
def tokenize(text):
    text = re.sub(r'[^a-zA-Z0-9\s]',' ',text)
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    cleaned_tokens = []
    for tok in tokens:
        clean_tok = WordNetLemmatizer().lemmatize(tok,).lower().strip()
        cleaned_tokens.append(clean_tok)
        
    return cleaned_tokens
        
    

In [144]:
toktest = tokenize(df['message'].iloc[0])
toktest

['weather',
 'update',
 'a',
 'cold',
 'front',
 'from',
 'cuba',
 'that',
 'could',
 'pas',
 'over',
 'haiti']

In [145]:
nas = df.isna().sum()
nas[nas>0]

original    16046
dtype: int64

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [146]:
pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize,token_pattern=None)),
                    ('Tfidf',TfidfTransformer()),
                    ('rf',MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [147]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [148]:
#this takes some time to run. Over 30 minutes. 
pipeline.fit(X_train,y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(token_pattern=None,
                                 tokenizer=<function tokenize at 0x000001983FE430D0>)),
                ('Tfidf', TfidfTransformer()),
                ('rf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [150]:
#saving the pipeline model 
import joblib
joblib.dump(pipeline, 'random_forest_model1.pkl')

['random_forest_model1.pkl']

In [151]:
#for reference. shape of y_test
y_test.shape

(5244, 36)

In [152]:
#run test then show first 5 
y_pred = pipeline.predict(X_test)
y_pred[:5]
#y_pred = pd.DataFrame(y_pred)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [153]:
#no surprise, this is the same shape as y_test
y_pred.shape

(5244, 36)

In [154]:
#turn y_pred into a dataframe then look at that. 
ycols = y_test.columns.tolist()
y_pred2 = pd.DataFrame(y_pred,columns=ycols)
#y_test[ycols[i]]
type(y_pred2)

pandas.core.frame.DataFrame

In [155]:
y_pred2.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [156]:
#sanity check that all the columns are the same on both sides 
print(y_test.columns)
print(y_pred2.columns)
print(set(y_test.columns)-set(y_pred2.columns))

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')
Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_r

In [157]:
#note to self: need a division by zero catch. 
l = []
for i in range(len(ycols)):
    
    accuracy=accuracy_score(y_test[ycols[i]],y_pred2[ycols[i]])
    precision=precision_score(y_test[ycols[i]],y_pred2[ycols[i]],average='weighted',zero_division=1)
    recall=recall_score(y_test[ycols[i]],y_pred2[ycols[i]],average='weighted')
    f1score = f1_score(y_test[ycols[i]],y_pred2[ycols[i]],average='weighted')    
    temp ={'Column':[ycols[i]]
                     ,'Accuracy':[accuracy]
                     ,'Precision':[precision],
                     'Recall':[recall],
                      'F1 Score':[f1score]}
    
    temp = pd.DataFrame(temp)
    #temp = {k:[v] for k,v in temp.items()}  
    l.append(temp)
print(len(l))
dfmetrics=pd.concat(l,axis=0)
dfmetrics


36


,Column,Accuracy,Precision,Recall,F1 Score
0,related,0.797101,0.787681,0.797101,0.761437
0,request,0.888063,0.886903,0.888063,0.870900
0,offer,0.996377,0.996390,0.996377,0.994569
0,aid_related,0.779939,0.782556,0.779939,0.774324
0,medical_help,0.923722,0.914516,0.923722,0.892165
0,medical_products,0.950610,0.939600,0.950610,0.931129
0,search_and_rescue,0.972159,0.972935,0.972159,0.959327
0,security,0.979977,0.960725,0.979977,0.970256
0,military,0.968535,0.959367,0.968535,0.954646
0,child_alone,1.000000,1.000000,1.000000,1.000000


In [158]:
#looking at average accuracy, precision, recalls, F1 scores since over 30 output variables. 
dfmetrics.describe()

,Accuracy,Precision,Recall,F1 Score
count,36.000000,36.000000,36.000000,36.000000
mean,0.946701,0.939557,0.946701,0.932822
std,0.053154,0.055230,0.053154,0.060269
min,0.779939,0.782556,0.779939,0.761437
25%,0.934211,0.917815,0.934211,0.917071
50%,0.959382,0.950484,0.959382,0.944646
75%,0.982742,0.982155,0.982742,0.974949
max,1.000000,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [160]:
#theoretical attempts.... that I abandoned. 
# #making a class that pulls attributes from GridSearch since the MultiOUtputClassifier is one step removed. 
# class MultiOutputGridSearchCV(GridSearchCV):
#     #note I'm pulling attributes from the GridSearchCV to be able to use for the MultiOutput. Hence the super command. 
#     def __init__(self, pipeline, param_grid, cv=5):
#         self.pipeline = pipeline 
#         super().__init__(pipeline.steps[-1][1], param_grid, cv=cv)

#     def fit(self, X, y=None):
#         self.best_index_ = 0
#         return super().fit(X, y)
        
# #even though these are not "that many", it's still 16 combinations. going to take some time.. 
# param_grid = {
#     'estimator__max_depth': [ 3, 5],
#     'estimator__min_samples_split': [2,  6],
#    'estimator__min_df':[1,2] ,
#     'estimator__max_df':[0.99,0.95]}

##########################################################################
#ideas from stack overflow... ... using all will be computationally expensive. 
# pipe_lr = Pipeline([('scl', StandardScaler()),
#                     ('LR', LogisticRegression(random_state=42))])pipe_dt = Pipeline([('scl', StandardScaler()),
#                     ('DT',DecisionTreeClassifier(random_state=42))])pipe_rf = Pipeline([('scl', StandardScaler()),
#                     ('RF',RandomForestClassifier(random_state=42))])pipe_knn = Pipeline([('scl', StandardScaler()),
#                     ('KNN', KNeighborsClassifier())])pipe_svm = Pipeline([('scl', StandardScaler()),
#                      ('SVM', svm.SVC(random_state=42))])pipe_xgb = Pipeline([('scl', StandardScaler()),
#                      ('XGB', XGBClassifier(random_state=42))])
# param_range = [1, 2, 3, 4, 5, 6]
# param_range_fl = [1.0, 0.5, 0.1]
# n_estimators = [50,100,150]
# learning_rates = [.1,.2,.3]lr_param_grid = [{'LR__penalty': ['l1', 'l2'],
#                    'LR__C': param_range_fl,
#                    'LR__solver': ['liblinear']}]dt_param_grid = [{'DT__criterion': ['gini', 'entropy'],
#                    'DT__min_samples_leaf': param_range,
#                    'DT__max_depth': param_range,
#                    'DT__min_samples_split': param_range[1:]}]rf_param_grid = [{'RF__min_samples_leaf': param_range,
#                    'RF__max_depth': param_range,
#                    'RF__min_samples_split': param_range[1:]}]knn_param_grid = [{'KNN__n_neighbors': param_range,
#                    'KNN__weights': ['uniform', 'distance'],
#                    'KNN__metric': ['euclidean', 'manhattan']}]svm_param_grid = [{'SVM__kernel': ['linear', 'rbf'], 
#                     'SVM__C': param_range}]
# lr_grid_search = GridSearchCV(estimator=pipe_lr,
#         param_grid=lr_param_grid,
#         scoring='accuracy',
#         cv=3)dt_grid_search = GridSearchCV(estimator=pipe_dt,
#         param_grid=dt_param_grid,
#         scoring='accuracy',
#         cv=3)rf_grid_search = GridSearchCV(estimator=pipe_rf,
#         param_grid=rf_param_grid,
#         scoring='accuracy',
#         cv=3)knn_grid_search = GridSearchCV(estimator=pipe_knn,
#         param_grid=knn_param_grid,
#         scoring='accuracy',
#         cv=3)svm_grid_search = GridSearchCV(estimator=pipe_svm,
#         param_grid=svm_param_grid,
#         scoring='accuracy',
#         cv=3)xgb_grid_search = GridSearchCV(estimator=pipe_xgb,
#         param_grid=xgb_param_grid,
#         scoring='accuracy',
#         cv=3)

# grids = [lr_grid_search, dt_grid_search, rf_grid_search, knn_grid_search, svm_grid_search, xgb_grid_search]for pipe in grids:
#     pipe.fit(X_train,y_train)

# grid_search = MultiOutputGridSearchCV(pipeline, param_grid)


# grid_dict = {0: 'Logistic Regression', 1: 'Decision Trees', 
#              2: 'Random Forest', 3: 'K-Nearest Neighbors', 
#              4: 'Support Vector Machines', 5: 'XGBoost'}for i, model in enumerate(grids):
#     print('{} Test Accuracy: {}'.format(grid_dict[i],
#     model.score(X_test,y_test)))
#     print('{} Best Params: {}'.format(grid_dict[i],          model.best_params_))

###################################################################################################

#cv = grid_search.fit(X_train, y_train)
# print(cv.best_params_)


In [172]:
'''for reference 
pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize,token_pattern=None)),
                    ('Tfidf',TfidfTransformer()),
                    ('rf',MultiOutputClassifier(RandomForestClassifier()))])
'''


#new idea... random search instead of trying every combination  which would take a LONG TIME 
# search_space = {
#     'vect__max_df': (0.9, 0.95, 0.99),
#     'vect__min_df': (0.01, 0.05, 0.1),
# 'rf__bootstrap': [True, False],
#  #'rf__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
#  'rf__max_features': ['auto', 'sqrt'],
#  'rf__min_samples_leaf': [1, 2, 4],
#  'rf__min_samples_split': [2, 5, 10],
#  'rf__n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000] }




vect_max_df = [0.9, 0.95, 0.99]
vect_min_df = [0.01, 0.05, 0.1]
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid

random_grid = {
            'vect__max_df': vect_max_df,
            'vect__min_df': vect_min_df,
            'rf__estimator__n_estimators': n_estimators,
            'rf__estimator__max_features': max_features,
            'rf__estimator__max_depth': max_depth,
            'rf__estimator__min_samples_split': min_samples_split,
            'rf__estimator__min_samples_leaf': min_samples_leaf,
            'rf__estimator__bootstrap': bootstrap,
               }
# Random search of parameters, using 3 fold cross validation, 
# doing 5 random combos and start the program before going to sleep.  
rf_random = RandomizedSearchCV(estimator = pipeline, param_distributions = random_grid, n_iter = 5, cv = 3, verbose=2, random_state=42, n_jobs = -1)


rf_random.fit(X_train, y_train)




Fitting 3 folds for each of 5 candidates, totalling 15 fits


C:\ProgramData\anaconda3\envs\analytics3_8\lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('vect',
                                              CountVectorizer(token_pattern=None,
                                                              tokenizer=<function tokenize at 0x000001983FE430D0>)),
                                             ('Tfidf', TfidfTransformer()),
                                             ('rf',
                                              MultiOutputClassifier(estimator=RandomForestClassifier()))]),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'rf__estimator__bootstrap': [True,
                                                                     False],
                                        'rf__estimator__max_depth': [10, 20, 30,
                                                                     40, 50, 60,
                                                                     70, 80, 90,
                                                                     100, 110,
                                                                     None],
                                        'rf__estimator__max_features': ['auto',
                                                                        'sqrt'],
                                        'rf__estimator__min_samples_leaf': [1,
                                                                            2,
                                                                            4],
                                        'rf__estimator__min_samples_split': [2,
                                                                             5,
                                                                             10],
                                        'rf__estimator__n_estimators': [100,
                                                                        311,
                                                                        522,
                                                                        733,
                                                                        944,
                                                                        1155,
                                                                        1366,
                                                                        1577,
                                                                        1788,
                                                                        2000],
                                        'vect__max_df': [0.9, 0.95, 0.99],
                                        'vect__min_df': [0.01, 0.05, 0.1]},
                   random_state=42, verbose=2)

In [173]:
#here are the best parameters from the random grid search. I only used 15 combinations and that ran overnight. 
#the number could be increased if you have the time. 
rf_random.best_params_

{'vect__min_df': 0.01,
 'vect__max_df': 0.9,
 'rf__estimator__n_estimators': 1155,
 'rf__estimator__min_samples_split': 5,
 'rf__estimator__min_samples_leaf': 2,
 'rf__estimator__max_features': 'sqrt',
 'rf__estimator__max_depth': 100,
 'rf__estimator__bootstrap': True}

In [176]:
# best = {'vect__min_df': [0.01],
#  'vect__max_df': [0.9],
#  'rf__estimator__n_estimators': [1155],
#  'rf__estimator__min_samples_split': [5],
#  'rf__estimator__min_samples_leaf': [2],
#  'rf__estimator__max_features': ['sqrt'],
#  'rf__estimator__max_depth': [100],
#  'rf__estimator__bootstrap': [True]} 
# best = pd.DataFrame(best)
# best

In [177]:
help(rf_random.best_score_)

Help on float64 object:

class float64(floating, builtins.float)
 |  float64(x=0, /)
 |  
 |  Double-precision floating-point number type, compatible with Python `float`
 |  and C ``double``.
 |  
 |  :Character code: ``'d'``
 |  :Canonical name: `numpy.double`
 |  :Alias: `numpy.float_`
 |  :Alias on this platform (win32 AMD64): `numpy.float64`: 64-bit precision floating-point number type: sign bit, 11 bits exponent, 52 bits mantissa.
 |  
 |  Method resolution order:
 |      float64
 |      floating
 |      inexact
 |      number
 |      generic
 |      builtins.float
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __abs__(self, /)
 |      abs(self)
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __divmod__(self, value, /)
 |      Return divmod(self, value).
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __float__(self, /)
 |      float(self)
 |  
 |  __floordiv__(self, value, /)
 

In [169]:
''' result: with a score of only .26, running 15 combinations is not enough. 
however, due to time constraints, sticking with 15 here (but not using the model resulting from this in the webpage). 
More could be run if you have a spare computer that is allowed to run 
 for whateever it takes to do a higher number of combos
 which I would bet would have some combos with better scores than 0.26 '''
best_svm = rf_random.best_estimator_
test_accuracy = best_svm.score(X_test, y_test)
test_accuracy

0.25877192982456143

In [174]:
#rgd my acronym for random grid search 
y_pred_rgs = rf_random.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [175]:
#note: need a division by zero catch. 
l = []
for i in range(len(ycols)):
    
    accuracy=accuracy_score(y_test[ycols[i]],y_pred_rgs[ycols[i]])
    precision=precision_score(y_test[ycols[i]],y_pred_rgs[ycols[i]],average='weighted',zero_division=1)
    recall=recall_score(y_test[ycols[i]],y_pred_rgs[ycols[i]],average='weighted')
    f1score = f1_score(y_test[ycols[i]],y_pred_rgs[ycols[i]],average='weighted')    
    temp ={'Column':[ycols[i]]
                     ,'Accuracy':[accuracy]
                     ,'Precision':[precision],
                     'Recall':[recall],
                      'F1 Score':[f1score]}
    
    temp = pd.DataFrame(temp)
    #temp = {k:[v] for k,v in temp.items()}  
    l.append(temp)
print(len(l))
dfmetrics=pd.concat(l,axis=0)
dfmetrics

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
#looking at average accuracy, precision, recalls since over 30 output variables. 
dfmetrics.describe()

In [ ]:
import sys 
sys.exit() 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
#the actual model is exported in the py script, not here in the "sketch" script
finalmodel = 'pending' 
#with open("emergency_type_classifier.pkl", "wb") as f:
#    pickle.dump(finalmodel, f)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
#I did